In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Social_Media_Analysis/final exam') #切換該目錄
os.listdir() #確認目錄內容

Mounted at /content/drive


['TaipeiSansTCBeta-Regular.ttf',
 'data',
 'dict',
 'final_exam_3.ipynb',
 'final_exam_2.ipynb',
 'net_func.py',
 'ex1_undirected_graph.html',
 'final_exam_1.ipynb']

In [3]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.8 MB/s eta 0:00:00


In [4]:
import time
from functools import reduce
from collections import Counter
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.matutils import corpus2csc, corpus2dense, Sparse2Corpus

import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
# 設定繁體中文詞庫
jieba.set_dictionary('./dict/dict.txt.big')

# 新增stopwords
with open('./dict/stopwords.txt',encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

In [5]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# 設定中文字體
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 資料前處理

In [7]:
df = pd.read_csv("./data/student_1_4_7.csv")  # 匯資料
df.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,artUrl,artContent
0,https://www.ptt.cc/bbs/Food/M.1716654489.A.6FD...,排餐料理吃到飽系列之--饗犇牛排館\n\n\n有圖有真相圖文並茂版：\nhttps://ni...
1,https://www.ptt.cc/bbs/Food/M.1716481596.A.614...,作者: elisa48 (詠夜月舞) 看板: Japan_Travel\n標題: [食記] ...
2,https://www.ptt.cc/bbs/Food/M.1716035313.A.D97...,餐廳名稱：馬路益燒肉飯\n 消費時間：2024.04\n 地址：澎湖縣馬公市光明路2...


In [9]:
# 只留下中文字
df["artContent"] = df["artContent"].str.replace("[^\u4e00-\u9fa5]+", "", regex=True)
df=df.rename(columns={'artContent': 'content'})
df.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,artUrl,content
0,https://www.ptt.cc/bbs/Food/M.1716654489.A.6FD...,排餐料理吃到飽系列之饗犇牛排館有圖有真相圖文並茂版用餐時間年月地點台北市中山區南京東路三段號...
1,https://www.ptt.cc/bbs/Food/M.1716481596.A.614...,作者詠夜月舞看板標題食記東京人形町時間多圖網誌版從人形町地鐵站到東京日本橋的路上眼角餘光被轉...
2,https://www.ptt.cc/bbs/Food/M.1716035313.A.D97...,餐廳名稱馬路益燒肉飯消費時間地址澎湖縣馬公市光明路號電話營業時間星期二公休每人平均價位可否刷...


In [11]:
# 設定繁體中文詞庫
jieba.set_dictionary("./dict/dict.txt.big")

# 新增stopwords
# jieba.analyse.set_stop_words('./dict/stop_words.txt') #jieba.analyse.extract_tags才會作用
with open("./dict/stopwords.txt", encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

# 設定斷詞 function
def getToken(row):
    seg_list = jieba.cut(row, cut_all=False)
    seg_list = [
        w for w in seg_list if w not in stopWords and len(w) > 1
    ]  # 篩選掉停用字與字元數大於1的詞彙
    return seg_list

df["words"] = df["content"].apply(getToken)
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Building prefix dict from /content/drive/MyDrive/Social_Media_Analysis/final exam/dict/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /content/drive/MyDrive/Social_Media_Analysis/final exam/dict/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u534bd08e2ebb5cc06e5d761945078578.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.u534bd08e2ebb5cc06e5d761945078578.cache
Loading model cost 2.916 seconds.
DEBUG:jieba:Loading model cost 2.916 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


,artUrl,content,words
0,https://www.ptt.cc/bbs/Food/M.1716654489.A.6FD...,排餐料理吃到飽系列之饗犇牛排館有圖有真相圖文並茂版用餐時間年月地點台北市中山區南京東路三段號...,"[排餐, 料理, 系列, 饗犇, 牛排館, 有圖, 真相, 圖文並茂, 用餐, 時間, 地點..."
1,https://www.ptt.cc/bbs/Food/M.1716481596.A.614...,作者詠夜月舞看板標題食記東京人形町時間多圖網誌版從人形町地鐵站到東京日本橋的路上眼角餘光被轉...,"[作者, 詠夜, 看板, 食記, 東京, 人形, 時間, 多圖, 網誌, 人形, 地鐵站, ..."
2,https://www.ptt.cc/bbs/Food/M.1716035313.A.D97...,餐廳名稱馬路益燒肉飯消費時間地址澎湖縣馬公市光明路號電話營業時間星期二公休每人平均價位可否刷...,"[餐廳, 名稱, 馬路, 燒肉, 消費, 時間, 地址, 澎湖縣, 馬公市, 光明, 路號,..."
3,https://www.ptt.cc/bbs/Food/M.1715262858.A.AF9...,餐廳名稱萊萊小籠喬艾消費時間年月地址中國上海市黄浦區天津路號營業時間圖文照片記錄上海吃蟹粉小...,"[餐廳, 名稱, 萊萊, 小籠, 喬艾, 消費, 時間, 地址, 中國, 上海市, 黄浦, ..."
4,https://www.ptt.cc/bbs/Food/M.1716909238.A.589...,餐廳名稱深圳機場深圳航空尊鵬閣貴賓室消費時間年年月圖文傍晚的深圳機場人少國際航廈也不大能逛的...,"[餐廳, 名稱, 深圳機場, 深圳, 航空, 尊鵬閣, 貴賓室, 消費, 時間, 圖文, 傍..."


# LDA處理

In [12]:
docs = df['words'].to_list()
docs[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['排餐',
 '料理',
 '系列',
 '饗犇',
 '牛排館',
 '有圖',
 '真相',
 '圖文並茂',
 '用餐',
 '時間',
 '地點',
 '台北市',
 '山區',
 '南京東路',
 '號樓',
 '電話',
 '價位',
 '自助',
 '排餐',
 '參考',
 '另加',
 '一成',
 '服務費',
 '環境',
 '位於',
 '南京',
 '建國',
 '交叉口',
 '飯店',
 '南京',
 '地點',
 '算是',
 '捷運',
 '松江',
 '南京站',
 '南京',
 '復興',
 '地帶',
 '交通',
 '上還',
 '算是',
 '便利',
 '地點',
 '餐廳',
 '位置',
 '處於',
 '二樓',
 '一旁',
 '入口',
 '二樓',
 '門口',
 '幾個',
 '座位',
 '當作',
 '是候',
 '位區',
 '進場',
 '自助式',
 '餐區',
 '地方',
 '則是',
 '規畫',
 '用餐',
 '區域',
 '空間',
 '說不上',
 '寬敞',
 '座位',
 '上還',
 '算是',
 '舒適',
 '算是',
 '適合',
 '小型',
 '聚餐',
 '聊天',
 '地方',
 '店面',
 '一樓',
 '入口處',
 '二樓',
 '門口',
 '櫃台',
 '用餐',
 '環境',
 '餐點',
 '餐桌',
 '餐具',
 '調味料',
 '提供',
 '牛排',
 '辣椒醬',
 '蕃茄醬',
 '研磨',
 '胡椒',
 '選擇',
 '飲料',
 '冰品',
 '提供',
 '餐包',
 '吐司',
 '兩款',
 '巧克力',
 '花生',
 '草莓',
 '醬料',
 '旁邊',
 '披薩',
 '浪費',
 '胃置',
 '蒸物區',
 '牛肉',
 '羅宋湯',
 '玉米濃湯',
 '湯碗',
 '沖泡式',
 '茶包',
 '提供',
 '天人',
 '系列',
 '香片',
 '綠茶',
 '烏龍茶',
 '醬料',
 '辣油',
 '醬油',
 '調味',
 '披薩',
 '檯面',
 '三盤',
 '披薩',
 '替換',
 '口味',
 '夏威夷',
 '臘腸',
 '泡菜',
 '燒肉',
 '味道',
 '以剛',
 '出爐

In [13]:
dictionary = Dictionary(docs)
# Filter out words that occur less than 5 documents, or more than 99% of the documents.
dictionary.filter_extremes(no_below=5, no_above=0.99)
print(dictionary)

Dictionary<2943 unique tokens: ['一成', '一旁', '一樓', '不錯', '二樓']...>


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
for idx, (k, v) in enumerate(dictionary.token2id.items()):
    print(f"{k}: {v}")
    if idx > 10:
        break

一成: 0
一旁: 1
一樓: 2
不錯: 3
二樓: 4
交通: 5
位於: 6
位置: 7
價位: 8
入口: 9
兩款: 10
冰品: 11


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# Bag-of-words representation of the documents.
# 用 gensim ldamodel input 需要將文章轉換成 bag of words
corpus = [dictionary.doc2bow(doc) for doc in docs]
type(corpus)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


list

### 訓練 topic model

In [16]:
ldamodel = LdaModel(
    corpus=corpus,
    id2word=dictionary, # 字典
    num_topics=4, # 生成幾個主題數
    random_state=2024, # 亂數
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


###  查看 LDA 主題代表字

In [17]:
ldamodel.print_topics(num_words=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.009*"晶片" + 0.008*"輝達" + 0.007*"中捷" + 0.007*"時間" + 0.007*"伺服器" + 0.006*"公司" + 0.006*"隨機" + 0.005*"美元" + 0.004*"市場" + 0.004*"美國"'),
 (1,
  '0.007*"時間" + 0.007*"美國" + 0.006*"股價" + 0.006*"公司" + 0.006*"輝達" + 0.005*"真的" + 0.005*"市場" + 0.005*"晶片" + 0.005*"伺服器" + 0.005*"台灣"'),
 (2,
  '0.007*"輝達" + 0.006*"捷運" + 0.006*"台灣" + 0.006*"隨機" + 0.005*"機器人" + 0.005*"時間" + 0.005*"市場" + 0.005*"公司" + 0.005*"億美元" + 0.005*"美國"'),
 (3,
  '0.008*"真的" + 0.007*"時間" + 0.006*"好吃" + 0.006*"台灣" + 0.005*"捷運" + 0.005*"搭配" + 0.005*"口感" + 0.005*"季後賽" + 0.005*"推薦" + 0.005*"日本"')]

### 視覺化呈現

In [18]:
pyLDAvis.enable_notebook()
p = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
p

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.061587 -0.001546       1        1  41.440647
2     -0.034303  0.008513       2        1  23.284876
0     -0.020885 -0.023656       3        1  18.770619
1     -0.006399  0.016690       4        1  16.503858, topic_info=     Term        Freq       Total Category  logprob  loglift
1174   輝達  301.000000  301.000000  Default  30.0000  30.0000
1085   公司  251.000000  251.000000  Default  29.0000  29.0000
2547   股價  197.000000  197.000000  Default  28.0000  28.0000
2012   晶片  302.000000  302.000000  Default  27.0000  27.0000
2551  伺服器  228.000000  228.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
1004   連結   29.281925  141.765401   Topic4  -5.9252   0.2244
201    名稱   30.109517  179.353890   Topic4  -5.8973   0.0171
111    地址   30.325704  208.835391   Topic4  -5.8902  -0.1280
133    日本   30.472627  229.072237   Topic4  -5.8853  -0.2156
17     口感   29.884704  216.274246   Topic4  -5.9048  -0.1776

[319 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1024      1  0.778176   一圈
1024      2  0.097272   一圈
1024      4  0.097272   一圈
2500      1  0.192520   下跌
2500      2  0.247526   下跌
...     ...       ...  ...
2262      4  0.088367   鵜鶘
424       1  0.764058   點的
424       2  0.066440   點的
424       3  0.066440   點的
424       4  0.132880   點的

[894 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])

從圖上來看，每個主題分得很清楚，但主題2,3,4的前三十主題字很相像，都有:輝達、台積電、股價、半導體等主題字<br>
主題2,3都有包含台灣的中捷隨機殺人事件關鍵字<br>
所以主題2,3,4的主題大概分別是中捷或輝達、科技、股票<br>
主題1應該就是美食主題，因為包含很多像好吃、口感、餐廳等主題字

In [19]:
pyLDAvis.save_html(p, "lda_zh.html")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
